In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Plot
plot_directory = 'result/plot_set'

In [3]:
# colors = [
#     '#e6194B',
#     '#f58231',
#     '#9A6324',
#     '#911eb4',
#     '#3cb44b',
#     '#f032e6',
#     '#4363d8',
# ]

In [4]:
colors = [
	'#FF0000',
	'#00FFFF',
	'#0000FF',
	'#00008B',
	'#ADD8E6',
	'#800080',
	'#7FFFD4',
	'#008000',
	'#FF00FF',
	'#FFC0CB',
	'#C0C0C0',
	'#FFA500',
	'#000000',
	'#800000',
]

In [5]:
def load_json_file(file_path):
	try:
		with open(file_path, 'r') as file:
			data = json.load(file)
		return data
	
	except Exception as e:
		print(f"An error occurred while loading the JSON file: {e}")
		return None

In [6]:
def extract_fpss(metric_list):
	return list(metric_list[list(metric_list.keys())[0]][0]['metric'].keys())

In [7]:
def to_accuracy_vector(accuracy_result_seq, fpss, type='F1'):
	accuracy_vector = []
	for fps in fpss:
		accuracy_vector.append(accuracy_result_seq[fps][type])
	
	return accuracy_vector

In [8]:
def plot_scatter(xs, ys, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	plt.scatter(xs, ys, c=colors[0], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [9]:
def plot_scatter_label(xs, ys, labels, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	for i in range(len(xs)):
		plt.scatter(xs[i], ys[i], c=colors[labels[i]], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [10]:
def filter_list_index(input_list, indices_to_remove):
	return [item for i, item in enumerate(input_list) if i not in indices_to_remove]

## Plot

In [11]:
plot_filenames = sorted(os.listdir(plot_directory))
plot_video_names = sorted(list(set([f.split('_')[0] for f in plot_filenames])))

In [12]:
fpss = extract_fpss(load_json_file(os.path.join(plot_directory, plot_video_names[0] + "_Accuracy_Result.json")))

In [13]:
omv_videos = []
acc_videos = []

for v in plot_video_names:
	omv_dict = {}
	for fps in fpss:
		omv_dict[fps] = []
	acc_list = []

	accuracy_result = load_json_file(os.path.join(plot_directory, v + "_Accuracy_Result.json"))
	movement_result = load_json_file(os.path.join(plot_directory, v + "_Movement_Result.json"))

	for class_idx in list(accuracy_result.keys()):
		for i in range(len(accuracy_result[class_idx])):
			accuracy_vector = to_accuracy_vector(accuracy_result[class_idx][i]['metric'], fpss)
			acc_list.append(accuracy_vector)

			for fps in fpss:
				movement_vector = movement_result[class_idx][i]['movement'][fps]
				omv_dict[fps].append(movement_vector)
	
	omv_videos.append(omv_dict)
	acc_videos.append(acc_list)

In [14]:
for i in range(len(plot_video_names)):
	video_name = plot_video_names[i]
	for j in range(len(omv_videos[0][fpss[0]][0])):
		for k in range(len(fpss)):
			fps = fpss[k]
			
			omv_fps = list(np.array(omv_videos[i][fps])[:, j])
			acc_fps = list(np.array(acc_videos[i])[:, k])

			# Remove Outliers
			outlier_index = [l for l in range(len(omv_fps)) if omv_fps[l] == -1]
			omv_fps_clean = filter_list_index(omv_fps, outlier_index)
			acc_fps_clean = filter_list_index(acc_fps, outlier_index)

			title = f'{video_name}; OMV Feature {j}; FPS: {fps}'

			correlation_matrix = np.corrcoef(omv_fps_clean, acc_fps_clean)
			correlation_coefficient = correlation_matrix[0, 1]
			print(f"{title} -> Corr: {correlation_coefficient}")
			# plot_scatter(omv_fps_clean, acc_fps_clean, 'OMV Feature', 'ACC', title)

		print("")

Video1; OMV Feature 0; FPS: 1 -> Corr: -0.02550863928181206
Video1; OMV Feature 0; FPS: 2 -> Corr: -0.027577359936741944
Video1; OMV Feature 0; FPS: 3 -> Corr: 0.009369864223430936
Video1; OMV Feature 0; FPS: 5 -> Corr: 0.15299368213325465
Video1; OMV Feature 0; FPS: 6 -> Corr: 0.23438343001361694
Video1; OMV Feature 0; FPS: 10 -> Corr: 0.24402603902119982
Video1; OMV Feature 0; FPS: 15 -> Corr: 0.24851861956906185
Video1; OMV Feature 0; FPS: 30 -> Corr: 0.24933952880573051

Video1; OMV Feature 1; FPS: 1 -> Corr: 0.012232081802778337
Video1; OMV Feature 1; FPS: 2 -> Corr: -0.0016201122019652609
Video1; OMV Feature 1; FPS: 3 -> Corr: 0.03875831859181694
Video1; OMV Feature 1; FPS: 5 -> Corr: 0.20211539635675468
Video1; OMV Feature 1; FPS: 6 -> Corr: 0.3019823473573191
Video1; OMV Feature 1; FPS: 10 -> Corr: 0.29813590470730833
Video1; OMV Feature 1; FPS: 15 -> Corr: 0.3118870499416207
Video1; OMV Feature 1; FPS: 30 -> Corr: 0.3193474311972535

Video1; OMV Feature 2; FPS: 1 -> Corr: -0.0